# Angestrebter Studienabschluss und Fachbereichsgröße

## Import

In [1]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

## Import Desing-Template

In [2]:
# Template
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

## Daten einlesen

In [ ]:
file_path = "data.xlsx"

# Read all sheets from Excel file
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Drop the first sheet (e.g. metadata)
sheets.pop(next(iter(sheets)))

cleaned_sheets = {}

for jahr, df in sheets.items():
    # Clean column names
    df.columns = df.columns.astype(str).str.strip()
    cleaned_sheets[jahr] = df

In [ ]:
abschluss_anteile = {}

for jahr, df in cleaned_sheets.items():
    # Identify relevant columns
    bachelor_columns = [col for col in df.columns if "Bachelor" in col]
    master_columns = [col for col in df.columns if "Master" in col]

    # Sum of enrolled students
    total_bachelor = df.loc[
        df["Category"] == "Anzahl Studierende", bachelor_columns
    ].sum().sum()
    total_master = df.loc[
        df["Category"] == "Anzahl Studierende", master_columns
    ].sum().sum()

    # Sum of survey participants
    total_ruecklauf_bachelor = df.loc[
        df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", bachelor_columns
    ].sum().sum()
    total_ruecklauf_master = df.loc[
        df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", master_columns
    ].sum().sum()

    # Compute totals and shares
    total_students = total_bachelor + total_master
    anteil_bachelor = (total_bachelor / total_students * 100) if total_students > 0 else 0
    anteil_master = (total_master / total_students * 100) if total_students > 0 else 0

    gesamt_ruecklauf = total_ruecklauf_bachelor + total_ruecklauf_master
    gesamt_ruecklaufquote = gesamt_ruecklauf / total_students * 100
    nicht_teilgenommen = total_students - gesamt_ruecklauf

    # Save results
    abschluss_anteile[jahr] = {
        "Absolute Anzahl Bachelor": total_bachelor,
        "Absolute Anzahl Master": total_master,
        "Absoluter Rücklauf Bachelor": total_ruecklauf_bachelor,
        "Absoluter Rücklauf Master": total_ruecklauf_master,
        "Gesamtzahl Studierende": total_students,
        "Gesamter Rücklauf": gesamt_ruecklauf,
        "Gesamte Rücklaufquote": gesamt_ruecklaufquote,
        "Gesamt nicht teilgenommen": nicht_teilgenommen,
        "Anteil Bachelor (%)": anteil_bachelor,
        "Anteil Master (%)": anteil_master,
    }

# Convert results to DataFrame
abschluss_df = (
    pd.DataFrame.from_dict(abschluss_anteile, orient="index")
    .reset_index()
    .rename(columns={"index": "Jahr"})
)

abschluss_df["Jahr"] = abschluss_df["Jahr"].astype(int)

## Entwicklung aller Abschlüsse

In [36]:
fig = px.line(
    abschluss_df,
    x="Jahr",
    y="Gesamte Rücklaufquote",
    markers=True,
    title="<span style='color:#73c6e9;'>Rücklaufquote</span> des Fachbereichs 09 nimmt seit Jahren ab",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Thicker line
fig.update_traces(line=dict(width=3))

# Set axis ranges and layout size
fig.update_layout(
    yaxis=dict(range=[0, 45], title="Rücklaufquote %"),
    xaxis=dict(range=[2012.9, 2024.2], dtick=1, title=""),
    height=450,
    width=1400,
)

fig.show()
fig.write_image("Plots/pdf/abschluss1.pdf")

In [37]:
color1, color2 = px.colors.qualitative.Safe[4], px.colors.qualitative.Safe[6]

fig = px.bar(
    abschluss_df,
    x="Jahr",
    y=["Gesamter Rücklauf", "Gesamt nicht teilgenommen"],
    title=rf"<b>Schrumpfen des Fachbereichs trifft </b>"
          f"<span style='color:{color1};'>Teilnehmende</span> <b>und </b>"
          f"<span style='color:{color2};'>nicht Teilnehmende</span>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=[color1, color2],
    template="infoviz"
)

# Layout configuration
fig.update_layout(
    yaxis=dict(range=[0, 3600], title="Anzahl Studierende"),
    xaxis=dict(dtick=1, title=""),
    showlegend=False,
    height=500,
    width=1400,
)

fig.show()
fig.write_image("Plots/pdf/abschluss2.pdf")

## Abschlussarten

In [38]:
fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absoluter Rücklauf Bachelor", "Absoluter Rücklauf Master"],
    markers=True,
    title="<b>Teilnehmeranzahl von </b><span style='color:#73c6e9;'>Bachelor</span> "
          "<b>und </b><span style='color:#cc5b6e;'>Master</span> "
          "<b>mittlerweile auf ähnlichem Niveau </b>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Style lines
fig.update_traces(line=dict(width=3))

# Layout settings
fig.update_layout(
    yaxis=dict(range=[0, 650], title="Anzahl Teilnehmende"),
    xaxis=dict(dtick=1, title="", range=[2012.9, 2024.2]),
    showlegend=False,
    height=500,
    width=1400,
)

fig.show()
fig.write_image("Plots/pdf/abschluss3.pdf")

In [39]:
fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absolute Anzahl Bachelor", "Absolute Anzahl Master"],
    markers=True,
    title="<b>Rückgang des Fachbereichs trifft </b><span style='color:#73c6e9;'>Bachelor</span> "
          "<b>härter als </b><span style='color:#cc5b6e;'>Master</span>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

# Style lines
fig.update_traces(line=dict(width=3))

# Layout settings
fig.update_layout(
    yaxis=dict(range=[0, 2100], title="Anzahl Studierende"),
    xaxis=dict(dtick=1, title="", range=[2012.9, 2024.2]),
    showlegend=False,
    height=500,
    width=1400,
)

fig.show()
fig.write_image("Plots/pdf/abschluss4.pdf")

## Verteilung der Studiengänge

In [40]:
df_2024 = cleaned_sheets["2024"]

# Identify Bachelor and Master columns
bachelor_studiengaenge = [col for col in df_2024.columns if "Bachelor" in col]
master_studiengaenge = [col for col in df_2024.columns if "Master" in col]

# Extract and transpose data
studiengang_verteilung = df_2024[
    df_2024["Category"] == "Anzahl Studierende"
][bachelor_studiengaenge + master_studiengaenge].T
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index

# Assign degree type 
studiengang_verteilung["Studienabschluss"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "Bachelor" in x else "Master"
)

# Clean program names
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: x.replace("Bachelor", "B.Sc.").replace("Master", "M.Sc.")
)

# shorten long program name
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].replace(
    "M.Sc. Informationstechnologie in den Agrar- und Umweltwissenschaften*",
    "M.Sc. Informationstechnologie"
)

# column for degree type based on cleaned name
studiengang_verteilung["Abschlussart"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "B.Sc." in x else "Master"
)

# Define color map
color_map = {
    "Bachelor": px.colors.qualitative.Safe[0],
    "Master": px.colors.qualitative.Safe[1]
}

# Sort by student count
studiengang_verteilung = studiengang_verteilung.sort_values(
    by="Anzahl Studierende",
    ascending=False
)

# Fix order for plotting
studiengang_verteilung["Studiengang"] = pd.Categorical(
    studiengang_verteilung["Studiengang"],
    categories=studiengang_verteilung["Studiengang"],
    ordered=True
)

In [41]:
fig = px.bar(
    studiengang_verteilung,
    x="Studiengang",
    y="Anzahl Studierende",
    title="Long-Tail in der Studierendenverteilung: Wenige große, viele kleine Studiengänge",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    color="Abschlussart",
    color_discrete_map=color_map,
    template="infoviz"
)

# Adjust layout: add bottom/right margin and axis formatting
fig.update_layout(
    margin=dict(b=190, r=130),
    xaxis=dict(title=""),
    height=600,
    width=1300,
    showlegend=True
)

fig.show()
fig.write_image("Plots/pdf/abschluss5.pdf")

In [42]:
# Reassign degree type based on cleaned name
studiengang_verteilung["Abschlussart"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "B.Sc." in x else "Master"
)

# Define custom color mapping
color_map = {
    "Bachelor": "rgb(136, 204, 238)",
    "Master": "rgb(204, 102, 119)"
}

# Sort and fix category order
studiengang_verteilung = studiengang_verteilung.sort_values(
    by="Anzahl Studierende",
    ascending=False
)

studiengang_verteilung["Studiengang"] = pd.Categorical(
    studiengang_verteilung["Studiengang"],
    categories=studiengang_verteilung["Studiengang"],
    ordered=True
)

# Create bar chart
fig = px.bar(
    studiengang_verteilung,
    x="Studiengang",
    y="Anzahl Studierende",
    color="Abschlussart",
    color_discrete_map=color_map,
    title="Long-Tail in der Studierendenverteilung: Wenige große, viele kleine Studiengänge",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    template="infoviz"
)

# Layout configuration
fig.update_layout(
    margin=dict(b=190, r=130),
    xaxis_title="",
    height=600,
    width=1300,
    xaxis=dict(
        categoryorder="array",
        categoryarray=studiengang_verteilung["Studiengang"]
    ),
    legend=dict(
        title="",
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="right",
        x=1
    )
)

fig.show()
fig.write_image("Plots/pdf/abschluss6.pdf")

In [43]:
# Ensure numeric values
studiengang_verteilung["Anzahl Studierende"] = pd.to_numeric(
    studiengang_verteilung["Anzahl Studierende"],
    errors="coerce"
).fillna(0)

# Identify top 4 programs
top4_studiengaenge = studiengang_verteilung.nlargest(4, "Anzahl Studierende")["Studiengang"].tolist()

# Color mapping: highlight top 4, others grey
color_map = {
    studiengang: px.colors.qualitative.Safe[i]
    for i, studiengang in enumerate(top4_studiengaenge)
}
color_map.update({
    studiengang: "lightgrey"
    for studiengang in studiengang_verteilung["Studiengang"]
    if studiengang not in top4_studiengaenge
})

# Create pie chart
fig = px.pie(
    studiengang_verteilung,
    values="Anzahl Studierende",
    names="Studiengang",
    title="Die vier größten Studiengänge machen über 50% aller Studierenden aus",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    color="Studiengang",
    color_discrete_map=color_map,
    template="infoviz",
    hole=0.2
)

# Layout configuration
fig.update_layout(
    font=dict(size=18),
    width=1400,
    height=800,
    showlegend=False,
    title=dict(
        font=dict(size=20, family="Arial", color="black"),
        xanchor="left",
        xref="container",
        x=0.08,
        subtitle=dict(
            font=dict(color="gray", size=13)
        )
    )
)

# Slice styling
fig.update_traces(
    textinfo="none",
    marker=dict(line=dict(color="black", width=1))
)

# Annotate individual segments
annotations = {
    "B.Sc. Ernährungswissenschaften": (0.72, 0.9),
    "B.Sc. Ökotrophologie": (0.17, 0.9),
    "M.Sc. Ernährungswissenschaften": (0.04, 0.5),
    "B.Sc. Agrarwissenschaften": (0.12, 0.2),
    "Andere": (0.73, 0.2)
}

for studiengang, (x_pos, y_pos) in annotations.items():
    if studiengang == "Andere":
        text = "Andere<br>1102 Studierende"
    else:
        value = studiengang_verteilung.loc[
            studiengang_verteilung["Studiengang"] == studiengang,
            "Anzahl Studierende"
        ].values[0]
        text = f"{studiengang}<br>{int(value)} Studierende"

    fig.add_annotation(
        text=text,
        x=x_pos,
        y=y_pos,
        showarrow=False,
        font=dict(size=14, color="black"),
        align="left",
        xanchor="left"
    )

fig.show()
fig.write_image("Plots/pdf/abschluss7.pdf")